<a href="https://colab.research.google.com/github/Intelligence-Engineering-LAB-KU/Deeplearning-Tutorial/blob/master/Tutorial_4_CIFAR10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. 라이브러리 로드

필요한 라이브러리를 로드해줍니다.

In [ ]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

## 2. 데이터셋 로드

이번에도 [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) 데이터를 사용할 예정입니다. 

CIFAR-10에 대한 데이터 살펴보기 예제는 [지난 튜토리얼](https://github.com/Intelligence-Engineering-LAB-KU/Deeplearning-Tutorial/blob/master/Tutorial_1_CIFAR10_Softmax.ipynb)를 복습해주세요.


In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

## 3. 클래스 이진코드화

[지난 튜토리얼](https://github.com/Intelligence-Engineering-LAB-KU/Deeplearning-Tutorial/blob/master/Tutorial_1_CIFAR10_Softmax.ipynb)에서와 같이, 클래스를 이진코드화 해보겠습니다.

In [ ]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

print('y_train shape (이진화 전)', y_train.shape)
print('y_train[0]:', y_train[0])

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train shape (이진화 후)', y_train.shape)
print('y_train[0]:', y_train[0])

y_train shape (이진화 전) (50000, 1)
y_train[0]: [6]
y_train shape (이진화 후) (50000, 10)
y_train[0]: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


### 4. CIFAR10 분류 CNN 모델 생성

4장에서는 Keras의 [Conv2d API](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) 등을 이용하여 CIFAR-10 분류를 위한 CNN 기반 모델을 정의해보겠습니다. 

```python
tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid', data_format=None,
    dilation_rate=(1, 1), activation=None, use_bias=True,
    kernel_initializer='glorot_uniform', bias_initializer='zeros',
    kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None,
    kernel_constraint=None, bias_constraint=None, **kwargs
)
```
#### 4.1. 모델 아키텍쳐

다음과 같은 모델을 구현해보겠습니다.

![](https://imgur.com/SDxHEhR.png)

In [ ]:
from tensorflow.keras.layers import MaxPool2D
MaxPool2D(pool_size=(2, 2))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.regularizers import l1

tf.random.set_seed(2020)

model = Sequential([
    Conv2D(filters=2,kernel_size=(3,3), padding='same', activation='relu', 
           input_shape=(32, 32, 3)),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=4,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=8,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=16,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=32,kernel_size=(3,3), padding='same', activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Conv2D(filters=64,kernel_size=(3,3), padding='same', activation='relu'),
    Flatten(),
    Dense(10, activation='softmax')])

## 5. 모델 학습

이제 모델을 학습시켜보겠습니다.

In [ ]:
tf.random.set_seed(2020)

opt = keras.optimizers.Adam()

# 모델 생성
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

norm_x_train = x_train.astype('float32')
norm_x_test = x_test.astype('float32')
norm_x_train /= 255
norm_x_test /= 255

model.fit(norm_x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(norm_x_test, y_test),
          shuffle=True)

Epoch 1/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.9076 - accuracy: 0.2666 - val_loss: 1.7090 - val_accuracy: 0.3555
Epoch 2/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.6619 - accuracy: 0.3758 - val_loss: 1.6284 - val_accuracy: 0.3920
Epoch 3/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.5942 - accuracy: 0.4089 - val_loss: 1.5536 - val_accuracy: 0.4243
Epoch 4/10
1563/1563 [==============================] - 37s 24ms/step - loss: 1.5433 - accuracy: 0.4329 - val_loss: 1.5119 - val_accuracy: 0.4448
Epoch 5/10
1563/1563 [==============================] - 38s 24ms/step - loss: 1.5067 - accuracy: 0.4478 - val_loss: 1.4775 - val_accuracy: 0.4590
Epoch 6/10
1563/1563 [==============================] - 39s 25ms/step - loss: 1.4768 - accuracy: 0.4628 - val_loss: 1.4609 - val_accuracy: 0.4634
Epoch 7/10
1563/1563 [==============================] - 36s 23ms/step - loss: 1.4505 - accuracy: 0.4747 - val_loss: 1.4300 -

## 6. 모델 성능 테스트

모델의 성능을 테스트해보겠습니다.

In [ ]:
scores = model.evaluate(norm_x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

313/313 [==============================] - 3s 11ms/step - loss: 1.4369 - accuracy: 0.5114
Test loss: 1.4369152784347534
Test accuracy: 0.5113999843597412


정확도가 50% 정도되는 간단한 모델을 만들어보았습니다.

## 7. 해보기

모델 부분을 수정하여서, 테스트 성능이 60% 이상 나오는 모델을 만들어보세요!

In [ ]:
tf.random.set_seed(2020)

##  이 부분을 수정하세요 ## 이 부분을 수정하세요 ## 이 부분을 수정하세요
my_model = #TODO-LIST
## 이 부분을 수정하세요 ## 이 부분을 수정하세요 ## 이 부분을 수정하세요

opt = keras.optimizers.Adam()

my_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

my_model.fit(norm_x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(norm_x_test, y_test),
          shuffle=True)

scores = my_model.evaluate(norm_x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Epoch 1/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.8882 - accuracy: 0.3448 - val_loss: 1.7552 - val_accuracy: 0.3921
Epoch 2/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.6867 - accuracy: 0.4188 - val_loss: 1.6567 - val_accuracy: 0.4231
Epoch 3/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.6314 - accuracy: 0.4386 - val_loss: 1.6256 - val_accuracy: 0.4391
Epoch 4/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.6053 - accuracy: 0.4476 - val_loss: 1.6128 - val_accuracy: 0.4436
Epoch 5/10
1563/1563 [==============================] - 27s 18ms/step - loss: 1.5919 - accuracy: 0.4519 - val_loss: 1.6034 - val_accuracy: 0.4430
Epoch 6/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.5816 - accuracy: 0.4566 - val_loss: 1.6066 - val_accuracy: 0.4446
Epoch 7/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.5739 - accuracy: 0.4594 - val_loss: 1.5984 -



## Reference

- [cs231n](http://cs231n.github.io/)
- [Keras tutorial on CIFAR-10](https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py)